Example of a function for reading .CLU files (PAJEK partitions) within networkx.

Using the style of networkx's 'read_pajek' at https://github.com/networkx/networkx/blob/master/networkx/readwrite/pajek.py

Once these functions are properly tested, I will propose a Pull Request to the networkx repository for that file

@Author: github.com/joaquincabezas

In [ ]:
import networkx as nx
from networkx.utils import open_file
from collections import defaultdict

In [ ]:
@open_file(0, mode='rb')
def read_pajek_communities(path, encoding='UTF-8'):
    """Read communities in Pajek format (.clu) from path.
    Parameters
    ----------
    path : file or string
       File or filename to read.
       Filenames ending in .gz or .bz2 will be uncompressed.
    Returns
    -------
    communities (list) – List of communities
    Examples
    --------
    >>> G = nx.read_pajek("simple.net")
    >>> communities = nx.read_pajek_communities("simple.clu")
    References
    ----------
    See http://vlado.fmf.uni-lj.si/pub/networks/pajek/doc/draweps.htm
    for format information.
    """
    lines = (line.decode(encoding) for line in path)
    return parse_pajek_communities(lines)

In [ ]:
def parse_pajek_communities(lines):
    """Parse Pajek format partition from string or iterable.
    Parameters
    ----------
    lines : string or iterable
       Data in Pajek partition format.
    Returns
    -------
    communities (list) – List of communities
    See Also
    --------
    read_pajek_partition()
    """
    if isinstance(lines, str):
        lines = iter(lines.split('\n'))
    lines = iter([line.rstrip('\n') for line in lines])
    
    while lines:
        try:
            l = next(lines)
        except:  # EOF
            break
        if l.lower().startswith("*vertices"):
            l, nnodes = l.split()
            communities = defaultdict(list)
            for vertex in range(int(nnodes)):
                l = next(lines)
                community = int(l)
                communities.setdefault(community, []).append(vertex)
        else:
            break
            
    return [ v for k,v in dict(communities).items() ]

In [ ]:
@open_file(1, mode='wb')
def write_pajek_communities(communities, path, encoding='UTF-8'):
    """Write partition in Pajek format (.clu) to path.
    Parameters
    ----------
    communities : list
       A communities list following networkx structure
    path : file or string
       File or filename to write.
       Filenames ending in .gz or .bz2 will be compressed.
    Examples
    --------
    >>> nx.write_pajek_communities(communities, "test.clu")
    Warnings
    --------
    We consider the vertices of the community regardless their precise numbering, only
    taking into account the ordinality
    References
    ----------
    See http://vlado.fmf.uni-lj.si/pub/networks/pajek/doc/draweps.htm
    for format information.
    """
    for line in generate_pajek_communities(communities):
        line += '\r\n'
        path.write(line.encode(encoding))

In [ ]:
def generate_pajek_communities(communities):
    """Generate lines in Pajek communities format (.clu).
    Parameters
    ----------
    communities : list
       A communities list
    References
    ----------
    See http://vlado.fmf.uni-lj.si/pub/networks/pajek/doc/draweps.htm
    for format information.
    """
    
    # We need a copy of the communities to use the strategy of removing vertices
    communities_list = [inner_list[:] for inner_list in communities]
    nnodes = sum( [ len(vertex) for vertex in communities])

    # Write first line
    yield f"*Vertices {nnodes}"
     
    # We do not assume that vertices:
    # - Starts with the number 1
    # - Are correlative
    # Therefore we will look for the minimum then pop it
    
    for n in range(0, nnodes):
        
        # We look for the minimum vertex number
        vertex =  min([min(item) for item in communities_list if item]) 
        
        # We find the community this vertex belongs to
        community = next(i for i,v in enumerate(communities_list) if vertex in v)
        
        # We put the community number in the row corresponding to the vertex
        # We add 1 because Pajek communities starts with number 1
        yield f"{community+1}"
        
        # We remove this vertex from the communities structure
        communities_list[community].remove(vertex)

In [ ]:
# Example 
communities_pajek = read_pajek_communities('simple.clu', encoding='UTF-8')

In [ ]:
write_pajek_communities(communities_pajek, "test.clu")

At this point, the two files: "simple.clu" and "test.clu" should be identical copies